In [8]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv

from skimage import io
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import PIL
import logging    # first of all import the module
import glob

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications import ResNet50, ResNet50V2, ResNet152V2
from tensorflow.keras.applications import InceptionV3, Xception

from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, fbeta_score, accuracy_score

os.chdir('/home/juanp.montoya/NeuralNetworks/Final_Project/final-project-landandbuildingsatimg-ccny/data/interim/Cropped')
os.getcwd()

'/home/juanp.montoya/NeuralNetworks/Final_Project/final-project-landandbuildingsatimg-ccny/data/interim/Cropped'

# Read Files in the Cropped Folder

In [9]:
data_df = pd.DataFrame(glob.glob('Data/*'), columns = ['train'])
mask_df = pd.DataFrame(glob.glob('Mask/*'), columns = ['mask'])

In [10]:
assert len(mask_df) == len(data_df)

In [11]:
nRowsRead = None 
df1 = pd.read_csv('class_dict.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'class_dict.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 7 rows and 4 columns


In [12]:
img_height = 612
img_width = 612

In [13]:
data_gen = ImageDataGenerator(rescale = 1. /255)
mask_gen = ImageDataGenerator()

batch_size = 32

train_generator = data_gen.flow_from_dataframe(dataframe=data_df, 
                                            directory="", 
                                            x_col="train", 
                                            class_mode=None,
                                            seed = 0,
                                            target_size=(img_height,img_width), 
                                            batch_size=batch_size)

mask_generator = mask_gen.flow_from_dataframe(dataframe=mask_df, 
                                            directory="", 
                                            x_col="mask", 
                                            class_mode=None,
                                            seed = 0,
                                            target_size=(img_height,img_width), 
                                            batch_size=batch_size)

gen = zip(train_generator, mask_generator)

KeyError: 'train'

In [19]:
def compile_model(input_shape, n_classes, optimizer, fine_tune=None):
    conv_base = ResNet50(include_top=False,
                                 weights='imagenet', 
                                 input_shape=input_shape)
    top_model = conv_base.output
    top_model = Flatten()(top_model)
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    model = Model(inputs=conv_base.input, outputs=output_layer)
            
    if type(fine_tune) == int:
        for layer in conv_base.layers[fine_tune:]:
            layer.trainable = True
    else:
        for layer in conv_base.layers:
            layer.trainable = False
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])
    
    return model

In [3]:
def plot_history(history):
       
    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc)
    plt.plot(val_acc)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    
    plt.subplot(1, 2, 2)
    plt.plot(loss)
    plt.plot(val_loss)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    
    plt.show();

In [4]:
def display_results(y_true, y_preds, class_labels):
    
    results = pd.DataFrame(precision_recall_fscore_support(y_true, y_preds),
                          columns=class_labels).T
    results.rename(columns={0: 'Precision',
                           1: 'Recall',
                           2: 'F-Score',
                           3: 'Support'}, inplace=True)
    
    conf_mat = pd.DataFrame(confusion_matrix(y_true, y_preds), 
                            columns=class_labels,
                            index=class_labels)    
    f2 = fbeta_score(y_true, y_preds, beta=2, average='micro')
    accuracy = accuracy_score(y_true, y_preds)
    print(f"Accuracy: {accuracy}")
    print(f"Global F2 Score: {f2}")    
    return results, conf_mat

# Plot Prediction Function

In [5]:
def plot_predictions(y_true, y_preds, test_generator, class_indices):

    fig = plt.figure(figsize=(20, 10))
    for i, idx in enumerate(np.random.choice(test_generator.samples, size=20, replace=False)):
        ax = fig.add_subplot(4, 5, i + 1, xticks=[], yticks=[])
        ax.imshow(np.squeeze(test_generator[idx]))
        pred_idx = np.argmax(y_preds[idx])
        true_idx = y_true[idx]
                
        plt.tight_layout()
        ax.set_title("{}\n({})".format(class_indices[pred_idx], class_indices[true_idx]),
                     color=("green" if pred_idx == true_idx else "red"))   

In [16]:
labels = df1.name.values

In [17]:
batch_size = 32
num_classes =len(labels)
input_shape = (img_height, img_height, 3)
class_mode = None
num_classes, input_shape

(7, (612, 612, 3))

# Running the ResNet50 Model

In [20]:
resnet50_model = compile_model(input_shape, num_classes, Adam(lr=1e-2), fine_tune=None)
resnet50_model.summary()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [76]:
batch = 0
x = train_generator[batch]
m = mask_generator[batch]

In [80]:
mask = m[1,:,:,:]
image = x[1,:,:,:]


In [83]:

N_STEPS = train_generator.samples//batch_size
# N_VAL_STEPS = test_generator.samples//batch_size
N_EPOCHS = 10
print(N_STEPS)
resnet50_history = resnet50_model.fit(zip(x, m))

401


ValueError: in user code:

    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **k

In [88]:

N_STEPS = train_generator.samples//batch_size
# N_VAL_STEPS = test_generator.samples//batch_size
N_EPOCHS = 10

resnet50_history = resnet50_model.fit(gen,
                             steps_per_epoch=N_STEPS,
                             epochs=N_EPOCHS,
                             )

Epoch 1/10


ValueError: in user code:

    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/juanp.montoya/anaconda3/envs/NN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **k

In [57]:

N_STEPS = train_generator.samples//batch_size
N_VAL_STEPS = test_generator.samples//batch_size
N_EPOCHS = 100

resnet50_history = resnet50_model.fit_generator(gen,
                             steps_per_epoch=N_STEPS,
                             epochs=N_EPOCHS,
                             callbacks=[early_stop, checkpoint, reduce_lr],
                             validation_data=test_generator,
                             validation_steps=N_VAL_STEPS)

NameError: name 'gen' is not defined

In [104]:
def segundoyultimo(lista):
    if lista[0] > lista[-1]:
        multiplicar = lista[0]*lista[-1]
        return (multiplicar)
    else: 
        suma = (lista[0] + lista[-1])
        return (suma)

In [105]:
lista = [0, 4, 2, 3, 4]
print(segundoyultimo(lista))

4


In [108]:
lista = [5, 2, 2, 3, 4]
print(segundoyultimo(lista))

20
